Building a Context-Aware Content Recommendation System Using BERT Embeddings |
News Articles Recommendations Using Sentence Transformers


Step - 1: Import required libraries

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\dell\OneDrive\Documents\PROJECTS\News Article Recommendations\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Step 2 : Load Your Dataset

In [2]:
df = pd.read_csv('Articles.csv', encoding='ISO-8859-1')
df

,Article,Date,Heading,NewsType
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business
...,...,...,...,...
2687,strong>DUBAI: Dubai International Airport and ...,3/25/2017,Laptop ban hits Dubai for 11m weekend traveller,business
2688,"strong>BEIJING: Former Prime Minister, Shaukat...",3/26/2017,Pak China relations not against any third coun...,business
2689,strong>WASHINGTON: Uber has grounded its fleet...,3/26/2017,Uber grounds self driving cars after accid,business
2690,strong>BEIJING: The New Development Bank plans...,3/27/2017,New Development Bank plans joint investments i...,business


Step 3 : Preprocessing the Text Data

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
# download
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
  #nConvert to lowercase
  text = text.lower()

  #Remove Punctuation
  text = re.sub(r'[^\w\s]', '', text)

  # Tokenize
  words = text.split()

  # remove stopwords and stemming
  preprocessed_text = [stemmer.stem(word) for word in words if word not in stop_words]

  return " ".join(preprocessed_text)



df['cleaned_heading'] = df['Heading'].apply(preprocess_text)

df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Article,Date,Heading,NewsType,cleaned_heading
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business,sindh govt decid cut public transport fare 7pc...
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business,asia stock new year trad
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business,hong kong stock open 066 percent lower
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business,asian stock sink euro near nine year
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business,us oil price slip 50 barr
...,...,...,...,...,...
2687,strong>DUBAI: Dubai International Airport and ...,3/25/2017,Laptop ban hits Dubai for 11m weekend traveller,business,laptop ban hit dubai 11m weekend travel
2688,"strong>BEIJING: Former Prime Minister, Shaukat...",3/26/2017,Pak China relations not against any third coun...,business,pak china relat third countri shaukat aziz
2689,strong>WASHINGTON: Uber has grounded its fleet...,3/26/2017,Uber grounds self driving cars after accid,business,uber ground self drive car accid
2690,strong>BEIJING: The New Development Bank plans...,3/27/2017,New Development Bank plans joint investments i...,business,new develop bank plan joint invest economi alo...


Step 4 : Initialize The Sentence Transformer Model

In [4]:
# load the pre-trained Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2') # You can use other models if you prefer

Step 5: Text Embeddings Using BERT

In [5]:
headings = df['cleaned_heading'].tolist()
embeddings = model.encode(headings, convert_to_tensor=True)

In [6]:
headings[0:5]

['sindh govt decid cut public transport fare 7pc kti rej',
 'asia stock new year trad',
 'hong kong stock open 066 percent lower',
 'asian stock sink euro near nine year',
 'us oil price slip 50 barr']

In [7]:
df.shape

(2692, 5)

In [8]:
embeddings[0:5]

tensor([[-0.0313,  0.0338, -0.0093,  ..., -0.0394,  0.0069,  0.0495],
        [-0.0520,  0.0016, -0.0221,  ..., -0.1266, -0.0875,  0.0417],
        [ 0.0940, -0.0362,  0.0582,  ..., -0.1183, -0.0391,  0.0702],
        [-0.0153,  0.0438,  0.0423,  ..., -0.0971, -0.0474,  0.0380],
        [-0.0803, -0.0079,  0.0873,  ..., -0.1084, -0.0170,  0.0278]])

Save and Load Embeddings

In [9]:
import pickle
pickle.dump(embeddings, open("embeddings.pkl", "wb"))

embeddings = pickle.load(open("embeddings.pkl", "rb"))

Step6: Recommendations

In [10]:
def recommend_articles_from_search(query, df, model, embeddings, num_recommendations=5):
  # Step 0: Clean Query
  query = preprocess_text(query) # Clean the query text (if needed)

  # Step 1: Get the embedding for the search query
  query_embedding = model.encode(query, convert_to_tensor=True)

  # Ensure query embeddings and article embeddings are on the CPU
  query_embedding = query_embedding.cpu()
  embeddings_cpu = embeddings.cpu()

  # Step 2: Calculate cosine similarity between query and all article embeddings
  similarities = cosine_similarity(query_embedding.reshape(1, -1), embeddings_cpu) # Query should be reshaped

  # Step 3: Get the indices of the top N most similar articles
  similarities = similarities.flatten()
  top_indices = similarities.argsort()[-num_recommendations:][::-1] # Get top N indices

  # Step 4: Return the corresponding articles and their similarity scores
  recommended_articles = df.iloc[top_indices][['Heading', 'NewsType', 'Article', 'Date']] # Adjust column

  return recommended_articles

# Example usage of the recommendation function
query = "Asian markets upswing"
recommended_articles = recommend_articles_from_search(query, df, model, embeddings, num_recommendations=10)

recommended_articles


,Heading,NewsType,Article,Date
119,most asian markets up tokyo at 15 year hig,business,Hong Kong: Japanese shares hit a 15-year high ...,4/22/2015
226,asian markets mostly recover from hefty sell off,business,Hong Kong: Most Asia shares rose Wednesday as ...,7/29/2015
2610,Dollar down Trump takes over Asia markets u,business,strong>HONG KONG: The dollar retreated against...,1/23/2017
220,asia markets mostly down shanghai up a 7th str...,business,Hong Kong: Asian markets mostly fell Friday fo...,7/24/2015
546,Asia stocks edge up to four month high after W...,business,strong>TOKYO: Asian shares edged up to a four-...,3/31/2016
307,asia markets rebound glencore surges after cr,business,Hong Kong: Asian stocks recovered slightly Wed...,9/30/2015
354,asia markets down after paris attacks japan in...,business,Hong Kong: Asian stock markets tumbled and the...,11/16/2015
432,Asia stocks cheered by China trade surpri,business,strong>SYDNEY: Asian shares made their first r...,1/13/2016
264,asian stocks up for second day on us gr,business,Hong Kong: Tokyo shares led Asian stock market...,8/28/2015
92,asian markets follow wall st lower after us d,business,Hong Kong: Asian markets mostly wound back Thu...,3/26/2015
